# GLOW - Contest Editors & Articles

## Table of Contents  <a class="anchor" id="toc"></a>

* [Contest editors & articles](#top)

    1. [Editors](#editors)
        1. [Editors](#editors_monthly)
        2. [Monthly Active Editors](#editors_active)
        3. [Monthly New Editors](#editors_new)
        4. [New editor retention](#new_editor_retention)
    2. [Articles](#articles)
        1. [Articles Count by wiki](#articles_count) 
        2. [New Articles](#new_articles)
        3. [Edits to existing articles](#article_edits)
        4. [New articles: by date/exp/survival](#new_articles_filtered)

In [1]:
#TO DO  https://github.com/wikimedia-research/canonical-data/blob/master/countries.csv

#https://better.engineering/2019/07/29/modeling-conversion-rates-and-saving-millions-of-dollars-using-kaplan-meier-and-gamma-distributions/

#investigating time spent by global wiki users https://opensym.org/wp-content/uploads/2019/08/os19-paper-A14-teblunthuis.pdf

#topics viz https://nbviewer.jupyter.org/github/conniecc1/topics-modeling/blob/master/TopicPageviewsSept2019.ipynb

In [19]:
import wmfdata as wmf
from wmfdata import charting, mariadb, hive
from wmfdata.utils import pct_str, pd_display_all
import requests
import re
from urllib import request
import json

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import jupyter_contrib_nbextensions
import datetime as dt 
from datetime import datetime, timedelta, date
import dateutil
#from dateutil.relativedelta import relativedelta

import logging
import gc
import weakref
import pprint

%load_ext sql_magic

The sql_magic extension is already loaded. To reload it, use:
  %reload_ext sql_magic


### Get clean article list: id, wiki, redirects <a class="anchor" id="get_clean_list"></a>
[Back to Table of Contents](#toc)

In [433]:
%store -r pa_articles_2018
%store -r pa_editors_2018

In [ ]:
#editors existing
#editors new
#editors active
#editors resource recipients

## Campaign new editors

In [454]:
editors_that_reg_b4_contest = wmf.mariadb.run("""
SELECT 
    DATE_FORMAT(user_registration,"%Y-%m-%d") AS reg_date
FROM user
WHERE
    user_name IN ({pa_editors_2018})
    AND user_registration >= "2018-03-01"
""".format(pa_editors_2018=pa_editors_2018), "pawiki")

## Campaign editor retention

In [486]:
#six months after contest ends, 2018-11-31
editor_activity = wmf.mariadb.run("""
SELECT 
    DATE_FORMAT(user_touched,"%Y-%m-%d") AS last_edit, 
    user_editcount
FROM user
WHERE
    user_name IN ({pa_editors_2018})
    AND user_touched >= "2018-11-31"
""".format(pa_editors_2018=pa_editors_2018), "pawiki")

In [ ]:
# six months after contest ends, 2018-11-31

## articles_expanded = first_edit_timestamp[(first_edit_timestamp['first_edited'] < '2018-11-01') | (first_edit_timestamp['first_edited'] > '2018-11-30') ]
#

editor_retention_nm_pa_editors_PT18 = wmf.mariadb.run("""
SELECT COUNT(DISTINCT user_id) AS editor_retention_pa_pt18
FROM user
JOIN revision 
on rev_user = user_id
WHERE user_name IN ({pa_editors_PT18})
  AND rev_timestamp >= "20181131000000"
  AND rev_timestamp <= "20190330000000"
  AND rev_minor_edit != 0
""".format(pa_editors_PT18=pa_editors_PT18), "pawiki")

In [ ]:
# editor_retention_nm_pa_editors_PT18

In [488]:
#six months after contest ends, 2018-11-31
editor_retention_all = wmf.mariadb.run("""
SELECT COUNT(DISTINCT user_id)
FROM user
JOIN revision 
on rev_user = user_id
WHERE user_name IN ({pa_editors_2018})
  AND rev_timestamp >= "2018-11-31"
""".format(pa_editors_2018=pa_editors_2018), "pawiki")

In [490]:
#six months after contest ends, 2018-11-31
editor_retention_nm = wmf.mariadb.run("""
SELECT COUNT(DISTINCT user_id)
FROM user
JOIN revision 
on rev_user = user_id
WHERE user_name IN ({pa_editors_2018})
  AND rev_timestamp >= "2018-11-31"
  AND rev_minor_edit != 0
""".format(pa_editors_2018=pa_editors_2018), "pawiki")

## Articles

In [ ]:
# articles count new
# articles count expanded
# articles count by resource recipients
# articles count by date/exp/survival

### New articles <a class="anchor" id="new_articles"></a>
[Back to Table of Contents](#toc)

In [ ]:
#TO DO use some of the queries in the content quality notebook to get counts here
%store -r articles_expanded
%store -r articles_new

In [498]:
nac_r_campaign_pt = hive.run("""
SELECT
    wiki_db, 
    count(DISTINCT page_title)
FROM wmf.mediawiki_history mh
WHERE
    mh.snapshot = "2019-09" 
    AND mh.event_timestamp >= "2018-03"
    AND event_entity = "page"
    AND event_type = "create"
    AND wiki_db = 'pawiki'
    AND page_title IN ({pa_articles_2018})
GROUP BY wiki_db
""".format(pa_articles_2018=pa_articles_2018))

### Topics & wikidata

In [ ]:
# READ DATA from Content quality and get aggregate totals for topics & wikidata
# top article categories
# num of pages created from topic lists
# top articles by pageviews